In [1]:
from dimod import (
    BinaryQuadraticModel
)

from dwave.system import LeapHybridSampler

import random
import pandas as pd
import numpy as np

# Define Token here to run
from credentials import TOKEN

from process_output import get_routes_from_sample, get_cost_routes, report_output, check_feasibility_sample

from vrp import VehicleRoutingProblem

In [2]:
def build_vrp_bqm(num_locations, distances, num_vehicles, max_distance):
    """
    Input:
        num_locations: number of distinct locations for orders
        distances: an matrix with the distances between respective locations for a fully connected graph
        num_vehicles: the number of vehicles used
        max_distance: the maximum distance that a truck can drive in one day
    Output:
        An unconstrained binary quadratic model with penalty terms to impose constraints representing a CVRP
    """

    M = num_vehicles
    N = num_locations

    # Penalty weights
    SINGLE_LOCATION_CONSTANT = 10**7
    VISIT_ALL_CONSTANT = 10**7

    # Create the binary variables and objective
    x, obj = VehicleRoutingProblem.construct_objective(num_locations, distances, num_vehicles)

    # Implement constraints as penalties to the objective
    
    # 1. Each location should be served by exactly one vehicle (only checks first N because depot is otherwise factored in)
    # TODO: Review again
    for k in range(N):
        # A term
        for a in range(M):
            for b in range(N):
                for c in range(a, M):
                    for d in range(N):
                        if a != b or c != d:
                            e = x[a, k, b]
                            f = x[c, k, d]
                            obj += x[a, k, b] * x[c, k, d] * VISIT_ALL_CONSTANT
                obj -= 0.5 * x[a, k, b] * VISIT_ALL_CONSTANT   

    # 2. Each vehicle is in at most one location (this accounts for how some paths will not hit all vertices)
    for m in range(M):
        for n in range(N):
            # A term
            for i in range(N+1):
                for j in range(i+1, N+1):
                    obj += SINGLE_LOCATION_CONSTANT * 2 * x[m, i, n] * x[m, j, n]
                obj -= SINGLE_LOCATION_CONSTANT * x[m, i, n]
                            
    # Return the unconstrained optimization solution
    return BinaryQuadraticModel(obj)

In [3]:
def run_bqm(bqm):
    """Run the provided BQM on the Leap Hybrid BQM Sampler with postselection of feasible solutions"""
    sampler = LeapHybridSampler(token=TOKEN)

    sampleset = sampler.sample(bqm)

    # TODO: Handle postselection
    return sampleset

    # feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

    # num_feasible = len(feasible_sampleset)
    # errors = " "
    # if num_feasible == 0:
    #     print("No feasible solution found.")
    #     return sampleset

    # print("\nFeasible solution found.\n")

    return feasible_sampleset

In [4]:
# Create a sample problem
num_destinations = 5
num_vehicles = 1
max_distance = 300

# Generate a random symmetric cost matrix
cost_matrix = [[0]*(num_destinations+1) for _ in range(num_destinations + 1)]

for i in range(num_destinations + 1):
    for j in range(i, num_destinations + 1):
        # Select random values that do not violate the triangle inequality
        if i == j:
            cost_matrix[i][j] = 0
        else:
            val = random.randint(5,9)
            cost_matrix[i][j] = val
            cost_matrix[j][i] = val

# Print the adjacency matrix
for row in cost_matrix:
    print(row)

[0, 9, 6, 7, 9, 7]
[9, 0, 6, 6, 6, 8]
[6, 6, 0, 8, 7, 5]
[7, 6, 8, 0, 6, 7]
[9, 6, 7, 6, 0, 6]
[7, 8, 5, 7, 6, 0]


### Test 1: Traveling Salesman Problem

In [5]:
bqm = build_vrp_bqm(num_destinations, cost_matrix, num_vehicles, max_distance)

In [6]:
feasible_sampleset = run_bqm(bqm)

In [7]:
print(feasible_sampleset)

  0_0_0 0_0_1 0_0_2 0_0_3 0_0_4 0_1_0 0_1_1 0_1_2 ... 0_5_4      energy num_oc.
0     1     0     0     0     0     0     0     0 ...     1 -54999986.0       1
['BINARY', 1 rows, 1 samples, 30 variables]


In [8]:
lowest_energy_sample = feasible_sampleset.lowest().first.sample

#lowest_energy_sample = {'0_0_0': 0, '0_0_1': 0, '0_0_2': 0, '0_0_3': 0, '0_0_4': 0, '0_1_0': 1, '0_1_1': 0, '0_1_2': 0, '0_1_3': 0, '0_1_4': 0, '0_2_0': 0, '0_2_1': 0, '0_2_2': 0, '0_2_3': 0, '0_2_4': 0, '0_3_0': 0, '0_3_1': 0, '0_3_2': 0, '0_3_3': 0, '0_3_4': 0, '0_4_0': 0, '0_4_1': 0, '0_4_2': 0, '0_4_3': 0, '0_4_4': 0, '0_5_0': 0, '0_5_1': 1, '0_5_2': 1, '0_5_3': 1, '0_5_4': 1}

routes = get_routes_from_sample(lowest_energy_sample, num_vehicles, num_destinations)

print(f'Best route: {routes}') # Nonsensical

#print(f'Best energy (with penalties): {feasible_sampleset.lowest().first.energy}')
print(f'Best cost: {get_cost_routes(routes, cost_matrix)}')
print(f'Feasible: {check_feasibility_sample(lowest_energy_sample, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True)}')

print(get_cost_routes(routes, cost_matrix))

#print(feasible_sampleset)

Best route: [[0]]
Best cost: 14
Violated constraint that location 1 must be visited
Feasible: False
14


### Test 2: Capacitated Multiple Vehicle Routing Problem

In [9]:
cqm2 = build_vrp_bqm(num_destinations, cost_matrix, num_vehicles=2, max_distance=30)

In [10]:
feasible_sampleset2 = run_bqm(cqm2)

In [17]:
lowest_energy_sample2 = feasible_sampleset2.lowest().first.sample

routes2 = get_routes_from_sample(lowest_energy_sample2, num_vehicles=2, num_steps=num_destinations)

report_output(routes2, cost_matrix)

print("Feasibility:")
print(f"{check_feasibility_sample(lowest_energy_sample2, num_vehicles, num_destinations, cost_matrix, max_distance, debug=True)}")

Best routes (depot omitted at start and end):
	Vehicle 0: [2]
	Vehicle 1: [1]
Best cost: 26
Feasibility:
Violated constraint that location 1 must be visited
False


In [12]:
# Lazy sanity check
r = []
for i in range(5):
    r.append(i)

min = 300
best_route = r
# Randomly sample a lot of options
for i in range(10000):
    random.shuffle(r)

    r1 = r[:2]
    r2 = r[2:]

    cost = get_cost_routes([r1, r2], cost_matrix)
    if cost < min:
        min = cost
        best_routes = [r1, r2]
print(min)
print(best_routes)

43
[[0, 2], [3, 1, 4]]
